<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Pooja/2021_CC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz

!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.download.n

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
#from pyspark import SparkFiles
#url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/2005_expenditure.csv"
#df2 = spark.read.options(inferSchema='True',delimiter=',', header=True).csv(url)
#df2.show()

from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Committee+Contributions+2021+(UF).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Committee+Contributions+2021+(UF).csv"), sep=",", header=True )
df.show()

+--------+--------+-------+--------+------------------+---------+------+--------+------+----------+--------+---------+----------+--------------------+------+-----+-------+---------+---------+------------+-----+-----+----------+-------+--------+----------+-------+--------+---------+---------+--------+----------+--------+------------+--------+--------+--------+--------+-----+------+--------------------+----------+-----------+----------+--------+-----------------+---------+--------+---------+------+-------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|     DATE|REFUNDDATE|                NAME|C_CODE|STRNO|STRNAME|APARTMENT|BOROUGHCD|        CITY|STATE|  ZIP|OCCUPATION|EMPNAME|EMPSTRNO|EMPSTRNAME|EMPCITY|EMPSTATE|     AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|INTERMNO|  INTERMNAME|INTSTRNO|INTSTRNM|INTAPTNO| INTCITY|INTST|INTZIP|          INTEMPNAME|INTEMPSTNO| INTEMPSTNM|INTEMPCITY|INTEMPST|        INTOCCUPA|PURPOSECD|EXEM

In [ ]:
df.dtypes

[('ELECTION', 'string'),
 ('OFFICECD', 'string'),
 ('RECIPID', 'string'),
 ('CANCLASS', 'string'),
 ('RECIPNAME', 'string'),
 ('COMMITTEE', 'string'),
 ('FILING', 'string'),
 ('SCHEDULE', 'string'),
 ('PAGENO', 'string'),
 ('SEQUENCENO', 'string'),
 ('REFNO', 'string'),
 ('DATE', 'string'),
 ('REFUNDDATE', 'string'),
 ('NAME', 'string'),
 ('C_CODE', 'string'),
 ('STRNO', 'string'),
 ('STRNAME', 'string'),
 ('APARTMENT', 'string'),
 ('BOROUGHCD', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('OCCUPATION', 'string'),
 ('EMPNAME', 'string'),
 ('EMPSTRNO', 'string'),
 ('EMPSTRNAME', 'string'),
 ('EMPCITY', 'string'),
 ('EMPSTATE', 'string'),
 ('AMNT', 'string'),
 ('MATCHAMNT', 'string'),
 ('PREVAMNT', 'string'),
 ('PAY_METHOD', 'string'),
 ('INTERMNO', 'string'),
 ('INTERMNAME', 'string'),
 ('INTSTRNO', 'string'),
 ('INTSTRNM', 'string'),
 ('INTAPTNO', 'string'),
 ('INTCITY', 'string'),
 ('INTST', 'string'),
 ('INTZIP', 'string'),
 ('INTEMPNAME', 'string'),
 (

In [ ]:
# Remove multiple columns
list = ['COMMITTEE', 'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY','REFUNDDATE','INTERMNO', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df = df.drop(*list)
df.show()

+--------+--------+-------+--------+------------------+---------+--------------------+------+---------+------------+-----+-----+--------+---------+---------+--------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|     DATE|                NAME|C_CODE|BOROUGHCD|        CITY|STATE|  ZIP|EMPSTATE|     AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|
+--------+--------+-------+--------+------------------+---------+--------------------+------+---------+------------+-----+-----+--------+---------+---------+--------+----------+
|    2013|       1|    204|       P|Quinn, Christine C|7/11/2011|1199 SEIU PAC - N...| PCOMP|        M|    New York|   NY|10036|    null|     1000|        0|       0|         2|
|    2013|       1|    204|       P|Quinn, Christine C|1/10/2013|1199 SEIU PAC - N...| PCOMP|        M|    New York|   NY|10036|    null|     3950|        0|    1000|         2|
|    2013|       1|    326|       P|   de Blasio, Bill|4/29/2010|1199/SEIU NYS Pol...| PCOMP|        M|    New

In [ ]:
df.select('RECIPID').distinct().show()

+-------+
|RECIPID|
+-------+
|   1669|
|     FI|
|    647|
|   1518|
|   1605|
|     AU|
|    326|
|   1796|
|    204|
|   1666|
|   1690|
|    260|
|     ED|
|     BB|
|   1757|
+-------+



In [ ]:
#df1 = df.filter(df.DATE.isNull())
#df1.select(when(df.DATE.isnull(),"0").alias("Date")).show()

TypeError: ignored

In [ ]:
#df1.show()

+--------+--------+-------+--------+---------------+----+--------------------+------+---------+-----------+-----+-----+--------+------+---------+--------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|      RECIPNAME|DATE|                NAME|C_CODE|BOROUGHCD|       CITY|STATE|  ZIP|EMPSTATE|  AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|
+--------+--------+-------+--------+---------------+----+--------------------+------+---------+-----------+-----+-----+--------+------+---------+--------+----------+
|    2013|       1|   1757|      NP| Person, Carl E|null|            123Print|  CORP|     null|  Frederick|   MD|21703|    null|115.95|        0|       0|         0|
|    2013|       1|   1757|      NP| Person, Carl E|null|3dna Corporation,...|  CORP|     null|Los Angeles|   CA|90013|    null|    19|        0|       0|         0|
|    2013|       1|   1757|      NP| Person, Carl E|null|       Dreamhost.com|  CORP|     null|       Brea|   CA|92821|    null| 49.75|        0|       0|         0|
|   

In [ ]:
df1 = df \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("RECIPID", "CandidateID") \
.withColumnRenamed("RECIPNAME", "CandidateName") \
.withColumnRenamed("DATE", "Date") \
.withColumnRenamed("NAME", "Name") \
.withColumnRenamed("C_CODE", "ContributerType") \
.withColumnRenamed("BOROUGHCD", "BoroughName") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "ZipCode") \
.withColumnRenamed("EMPSTATE", "ContributionState") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("MATCHAMNT", "MatchAmt") \
.withColumnRenamed("PREVAMNT", "PrevAmt") \
.withColumnRenamed("PAY_METHOD", "PayMethod") 

df1.printSchema()

root
 |-- Election: string (nullable = true)
 |-- OFFICECD: string (nullable = true)
 |-- CandidateID: string (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributerType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: string (nullable = true)
 |-- ContributionState: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- MatchAmt: string (nullable = true)
 |-- PrevAmt: string (nullable = true)
 |-- PayMethod: string (nullable = true)



In [ ]:
# changing datatypes of columns
df2 = df1 \
  .withColumn("Election",df1["Election"].cast("Integer")) \
  .withColumn("OFFICECD", df1["OFFICECD"].cast("Integer")) \
  .withColumn("CandidateID", df1["CandidateID"].cast("Integer")) \
  .withColumn("ZipCode", df1["ZipCode"].cast("Integer")) \
  .withColumn("Amount", df1["Amount"].cast("Float")) \
  .withColumn("MatchAmt", df1["MatchAmt"].cast("Float")) \
  .withColumn("PrevAmt", df1["PrevAmt"].cast("Float")) \
  
df2.printSchema()

root
 |-- Election: integer (nullable = true)
 |-- OFFICECD: integer (nullable = true)
 |-- CandidateID: integer (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributerType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- ContributionState: string (nullable = true)
 |-- Amount: float (nullable = true)
 |-- MatchAmt: float (nullable = true)
 |-- PrevAmt: float (nullable = true)
 |-- PayMethod: string (nullable = true)



In [ ]:
# converting column from string to datetime
#from datetime import datetime
#from pyspark.sql.functions import col, udf, when, unix_timestamp
#from pyspark.sql.types import DateType

#df2.select(when(df2['Date'].isNull(), "").df2['Date'].alias("New_Date")).collect()

#df2 = df2 \
#.withColumn("Date",when((df2.Date.isNull()) ,"")).show()
#.withColumn('Date_Formatted',when((df2["Date"].isNull() | (df2["Date"] == '')) ,'0')\
#.otherwise(to_date(str(df2.Date),"MM-dd-yyyy")))\
#.show()
#df2.select(df2["Date"],to_date(df2["Date"],"MM-dd-yyyy")).show()

AnalysisException: ignored

In [ ]:
#Change vaule name in ContributerType
from pyspark.sql.functions import regexp_replace
df3 = df2.withColumn('ContributerType', regexp_replace('ContributerType', 'CAN', 'Candidate')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'CORP', 'Corporation')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'EMPO', 'Labor Union')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'FAM', 'Candidate Family')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'IND', 'Individual')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMZ', 'Party Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'ORG', 'Orgainization')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'LLC', 'Limited Liability Company')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMP', 'Political Action Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'SPO', 'Spouse')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'OTHR', 'Other')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMC', 'Candidate Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PART', 'Individual')) \

In [ ]:
#Change Payment Method Name
df4=df3.withColumn('PayMethod', regexp_replace('PayMethod','0','Unknown')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','1','Cash')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','2','Check')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','3','Other')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','4','Credit Card')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','5','Money Order')) \

In [ ]:
#Change Borough name
from pyspark.sql.functions import regexp_replace
df5 = df4.withColumn('BoroughName', regexp_replace('BoroughName', 'K', 'Brooklyn')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'M', 'Manhattan')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Q', 'Queens')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'S', 'Staten Island')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'X', 'Bronx')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Z', 'Other')) \

In [ ]:
df6 = df5.filter((df5.OFFICECD=="1") & (df5.CANCLASS=="P"))
df6.show()

+--------+--------+-----------+--------+------------------+----+--------------------+--------------------+-----------+------------+-----+-------+-----------------+---------+--------+-------+---------+
|Election|OFFICECD|CandidateID|CANCLASS|     CandidateName|Date|                Name|     ContributerType|BoroughName|        City|State|ZipCode|ContributionState|   Amount|MatchAmt|PrevAmt|PayMethod|
+--------+--------+-----------+--------+------------------+----+--------------------+--------------------+-----------+------------+-----+-------+-----------------+---------+--------+-------+---------+
|    2013|       1|        204|       P|Quinn, Christine C|null|1199 SEIU PAC - N...|Political Action ...|  Manhattan|    New York|   NY|  10036|             null|   1000.0|     0.0|    0.0|    Check|
|    2013|       1|        204|       P|Quinn, Christine C|null|1199 SEIU PAC - N...|Political Action ...|  Manhattan|    New York|   NY|  10036|             null|   3950.0|     0.0| 1000.0|    Ch

In [ ]:
#Check the above dataframe to make sure only unique values between the OFFICECD is 1 
df6.select('OFFICECD').distinct().show()


+--------+
|OFFICECD|
+--------+
|       1|
+--------+



In [ ]:
#Drop OFFICECD and CANCLASS
list2 = ['OFFICECD', 'CANCLASS'] 
df7 = df6.drop(*list2)
df7.show()

+--------+-----------+------------------+----+--------------------+--------------------+-----------+------------+-----+-------+-----------------+---------+--------+-------+---------+
|Election|CandidateID|     CandidateName|Date|                Name|     ContributerType|BoroughName|        City|State|ZipCode|ContributionState|   Amount|MatchAmt|PrevAmt|PayMethod|
+--------+-----------+------------------+----+--------------------+--------------------+-----------+------------+-----+-------+-----------------+---------+--------+-------+---------+
|    2013|        204|Quinn, Christine C|null|1199 SEIU PAC - N...|Political Action ...|  Manhattan|    New York|   NY|  10036|             null|   1000.0|     0.0|    0.0|    Check|
|    2013|        204|Quinn, Christine C|null|1199 SEIU PAC - N...|Political Action ...|  Manhattan|    New York|   NY|  10036|             null|   3950.0|     0.0| 1000.0|    Check|
|    2013|        326|   de Blasio, Bill|null|1199/SEIU NYS Pol...|Political Action .

In [ ]:
# ADD AMNT , MATCHAMNT & PrevAmnt
from pyspark.sql.functions import col
df8 = df7.withColumn("TotalAmount", col("Amount")+col("MatchAmt")+col('PrevAmt'))
df8.show()

+--------+-----------+------------------+----+--------------------+--------------------+-----------+------------+-----+-------+-----------------+---------+--------+-------+---------+-----------+
|Election|CandidateID|     CandidateName|Date|                Name|     ContributerType|BoroughName|        City|State|ZipCode|ContributionState|   Amount|MatchAmt|PrevAmt|PayMethod|TotalAmount|
+--------+-----------+------------------+----+--------------------+--------------------+-----------+------------+-----+-------+-----------------+---------+--------+-------+---------+-----------+
|    2013|        204|Quinn, Christine C|null|1199 SEIU PAC - N...|Political Action ...|  Manhattan|    New York|   NY|  10036|             null|   1000.0|     0.0|    0.0|    Check|     1000.0|
|    2013|        204|Quinn, Christine C|null|1199 SEIU PAC - N...|Political Action ...|  Manhattan|    New York|   NY|  10036|             null|   3950.0|     0.0| 1000.0|    Check|     4950.0|
|    2013|        326|   

In [ ]:
df6.select('ContributerType').distinct().show()

In [ ]:
df6.select('PayMethod').distinct().show()

In [ ]:
df6.select('BoroughName').distinct().show()